In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib
import keras
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import SimpleRNN,LSTM,GRU,Embedding,Dense,Dropout,Input
from tensorflow.keras.optimizers import Adam,Nadam
from keras import Model

In [2]:
def tok_map(data):
    source = data['en'].values
    target = data['hi'].values
    target = '\t'+target+'\n'

    len_list_s = [len(i) for i in source]
    s_max_len = max(len_list_s)

    len_list_t = [len(i) for i in target]
    t_max_len = max(len_list_t)

    s_tok = set()
    for sw in source:
        for chr in sw:
            s_tok.add(chr)
    source_tokens = sorted(list(s_tok))
    s_tok_map = dict([(chr,i+1) for i,chr in enumerate(source_tokens)])
    s_tok_map[" "] = 0

    t_tok = set()
    for st in target:
        for chr in st:
            t_tok.add(chr)
    tar_tokens = sorted(list(t_tok))
    t_tok_map = dict([(chr,i+1) for i,chr in enumerate(tar_tokens)])
    t_tok_map[" "] = 0

    return source_tokens, s_tok_map, s_max_len, tar_tokens, t_tok_map, t_max_len

def dataLoad(path):
    with open(path) as dataFile:
        dataset = pd.read_csv(dataFile,sep='\t',header=None,names=["hi","en",""],skip_blank_lines=True,index_col=None)
    #print(dataset.head())
    dataset = dataset[dataset['hi'].notna()]
    #print(dataset.head())
    dataset = dataset[dataset['en'].notna()]
    #print(dataset.head())
    dataset = dataset[['hi','en']]
    #print(dataset.head())
    return dataset

def dataProcess(data):
    src,tar = data['en'].values, data['hi'].values
    tar = "\t" + tar + "\n"

    slen = len(src)
    enc_inp = np.zeros(
        (slen,s_max_len), dtype="float32"
    )

    tlen = len(tar)
    dec_inp = np.zeros(
        (tlen,t_max_len), dtype="float32"
    )
    dec_tar = np.zeros(
        (tlen, t_max_len, len(tar_tokens)+1), dtype="int"
    )
    for i,(sw,tw) in enumerate(zip(src,tar)):
        for j,ch in enumerate(sw):
            enc_inp[i,j] = s_tok_map[ch]
        enc_inp[i,j+1:] = s_tok_map[" "]

        for j,ch in enumerate(tw):
            dec_inp[i,j] = t_tok_map[ch]
            if j>0:
                dec_tar[i,j-1,t_tok_map[ch]] = 1
        dec_inp[i,j+1:] = t_tok_map[" "]
        dec_tar[i,j:,t_tok_map[" "]] = 1
        
    return enc_inp, dec_inp, dec_tar

In [3]:
train = dataLoad("/content/drive/MyDrive/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv")
source_tokens, s_tok_map, s_max_len, tar_tokens, t_tok_map, t_max_len = tok_map(train)
dev = dataLoad("/content/drive/MyDrive/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv")
test = dataLoad("/content/drive/MyDrive/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv")   

In [4]:
# Process the training data
train_encoder_input, train_decoder_input, train_decoder_target = dataProcess(train)

# Process the validation data
val_encoder_input, val_decoder_input, val_decoder_target = dataProcess(dev)

In [5]:
print(train)

              hi          en
0             अं          an
1        अंकगणित    ankganit
2           अंकल       uncle
3          अंकुर       ankur
4         अंकुरण     ankuran
...          ...         ...
44199  ह्वेनसांग  hiuentsang
44200  ह्वेनसांग  hsuantsang
44201  ह्वेनसांग    hyensang
44202  ह्वेनसांग    xuanzang
44203          ॐ          om

[44202 rows x 2 columns]


In [6]:
train.head()

,hi,en
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran


In [7]:
def seq2seqModel(Layer = "LSTM", nunits = 32, encl = 2, decl = 2,embds = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    enc_inps = Input(shape=(None,))
    enc_emb = Embedding(input_dim=len(source_tokens)+1, output_dim = embds, mask_zero=True)
    encop = enc_emb(enc_inps)

    dec_inps = Input(shape=(None,))
    dec_emb = Embedding(input_dim = len(tar_tokens)+1,output_dim = embds,mask_zero=True)

    # If the cell type is chosen as RNN ----------------------------------------------------
    if Layer == "RNN":
        encLays = []
        for i in range(encl-1):
            encLays.append(SimpleRNN(nunits,return_sequences=True))
        encLast = SimpleRNN(nunits,return_state=True)
        encmb = enc_emb(enc_inps)
        for enLay in encLays:
            encmb = enLay(encmb)
            if dropout is not None:
                encmb = Dropout(dropout)(encmb)

        _, state = encLast(encmb)
        encoder_states = state
        
        decoder = []
        for i in range(decl):
            decoder.append(SimpleRNN(nunits,return_sequences=True,return_state=True))

        for i in range(decl):
            if(i==0):
                dLhInp,_ = decoder[0](dec_emb(dec_inps),initial_state=state)
            else:
                dLhInp,_ = decoder[i](dLhInp,initial_state=state)

    # If the cell type is chosen as LSTM ----------------------------------------------------    
    elif Layer == "LSTM":
        encLays = []
        for i in range(encl-1):
            encLays.append(LSTM(nunits,return_sequences=True))
        encLast = LSTM(nunits,return_state=True)
        encmb = enc_emb(enc_inps)
        for enLay in encLays:
            encmb = enLay(encmb)
            if dropout is not None:
                encmb = Dropout(dropout)(encmb)
            
        _, state_h,state_c = encLast(encmb)
        encoder_states = [state_h,state_c]
        
        decoder = []
        for i in range(decl):
            decoder.append(LSTM(nunits,return_sequences=True,return_state=True))

        for i in range(decl):
            if(i==0):
                dLhInp,_,_ = decoder[0](dec_emb(dec_inps),initial_state=encoder_states)
            else:
                dLhInp,_,_ = decoder[i](dLhInp,initial_state=encoder_states)

    # If the cell type is chosen as GRU ----------------------------------------------------  
    elif Layer == "GRU":
        encLays = []
        for i in range(encl-1):
            encLays.append(GRU(nunits,return_sequences=True))

        encLast = GRU(nunits,return_state=True)
        encmb = enc_emb(enc_inps)
        for enLay in encLays:
            encmb = enLay(encmb)
            if dropout is not None:
                encmb = Dropout(dropout)(encmb)
            
        _, state = encLast(encmb)
        encoder_states = state
        
        decoder = []
        for i in range(decl):
            decoder.append(GRU(nunits, return_sequences=True, return_state=True))
        
        for i in range(decl):
            if(i==0):
                dLhInp,_ = decoder[0](dec_emb(dec_inps),initial_state=state)
            else:
                dLhInp,_ = decoder[i](dLhInp,initial_state=state)
            
        
    DLayerH = Dense(dense_size, activation='relu')
    DL_O = Dense(len(tar_tokens)+1, activation = 'softmax')
    act_op = DL_O(DLayerH(dLhInp))
    
    train_model = Model([enc_inps,dec_inps],act_op)

    return train_model

In [8]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'lr': {
            'values': [0.005, 0.001]
        },
        'epochs' : {
            'values': [15,20]
        },
        'nunits': {
            'values': [128, 256, 512]
        },
        'cell': {
            'values': ["LSTM", "GRU", "RNN"]
        },
        'emb_sz': {
            'values': [32, 64, 128, 256]
        },
        'edls': {
            'values': [2, 3, 4]
        },
        'dense_sz': {
            'values': [64, 128, 512]
        },
        'dp': {
            'values': [0.0,0.25,0.4]
        }
    }
}

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.callbacks import EarlyStopping

def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 15,
        'lr': 1e-2,
        'dense_sz': 128,
        'nunits': 128,
        'cell': 'LSTM',
        'emb_sz': 64,
        'edls': 1,
        'dp': 0.0
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults,name="cs6910-AS3-PA")
    cfg = wandb.config
    
    wandb.run.name = f'ep_{cfg.epochs}_un_{cfg.nunits}_cell_{cfg.cell}_emb_{cfg.emb_sz}_edls_{cfg.edls}_ds_{cfg.dense_sz}_do_{cfg.dp}_lr_{cfg.lr}'
    wandb.run.save()

    #Building the model using the parameters provided by the config ---------------------------------------    
    train = seq2seqModel(Layer=cfg.cell, nunits=cfg.nunits, embds=cfg.emb_sz, encl=cfg.edls, decl=cfg.edls, dense_size=cfg.dense_sz, dropout=cfg.dp)
    train.compile(optimizer = Adam(learning_rate=cfg.lr),loss='categorical_crossentropy',metrics=['accuracy'])
    print("Build Sucessfully")

    # Define early stopping mechanism to reduce extra useless epochs
    earlystopping = EarlyStopping(
        monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto"
    )

    # Fit the model after sucessfull model building 
    train.fit([train_encoder_input,train_decoder_input],train_decoder_target,
             batch_size=64,
             validation_data = ([val_encoder_input,val_decoder_input],val_decoder_target),
             epochs=cfg.epochs,
             callbacks = [earlystopping, WandbCallback(monitor='val_accuracy',mode='max')])
    print("Completed ! ")

In [ ]:
sweep_id = wandb.sweep(sweep_config,entity="pandp",project = 'CS6910-AS3')

Create sweep with ID: 3sng050k
Sweep URL: https://wandb.ai/pandp/CS6910-AS3/sweeps/3sng050k


In [ ]:
sweep_id = "3sng050k"

Create sweep with ID: 3sng050k

Sweep URL: https://wandb.ai/pandp/CS6910-AS3/sweeps/3sng050k

In [ ]:
wandb.agent(sweep_id, train,entity="pandp",project = 'CS6910-AS3')

wandb: Agent Starting Run: su328vnh with config:
wandb: 	cell: LSTM
wandb: 	dense_sz: 512
wandb: 	dp: 0.4
wandb: 	edls: 3
wandb: 	emb_sz: 64
wandb: 	epochs: 15
wandb: 	lr: 0.001
wandb: 	nunits: 512


Build Sucessfully
Epoch 1/15
691/691 [==============================] - 118s 134ms/step - loss: 0.8171 - accuracy: 0.4327 - val_loss: 0.4775 - val_accuracy: 0.6308 - _timestamp: 1651574868.0000 - _runtime: 134.0000
Epoch 2/15
691/691 [==============================] - 84s 121ms/step - loss: 0.3282 - accuracy: 0.7469 - val_loss: 0.2315 - val_accuracy: 0.8133 - _timestamp: 1651574951.0000 - _runtime: 217.0000
Epoch 3/15
691/691 [==============================] - 84s 121ms/step - loss: 0.1781 - accuracy: 0.8577 - val_loss: 0.1761 - val_accuracy: 0.8537 - _timestamp: 1651575035.0000 - _runtime: 301.0000
Epoch 4/15
691/691 [==============================] - 83s 120ms/step - loss: 0.1222 - accuracy: 0.9001 - val_loss: 0.1614 - val_accuracy: 0.8675 - _timestamp: 1651575118.0000 - _runtime: 384.0000
Epoch 5/15
691/691 [==============================] - 83s 121ms/step - loss: 0.0909 - accuracy: 0.9244 - val_loss: 0.1650 - val_accuracy: 0.8695 - _timestamp: 1651575201.0000 - _runtime: 467.0000
E

accuracy,▁▅▆▇▇██████████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▇████████████
val_loss,█▃▁▁▁▁▁▂▂▂▂▂▂▂▂
accuracy,0.9794
best_epoch,13
best_val_accuracy,0.87267
epoch,14
loss,0.02461
val_accuracy,0.86987


wandb: Agent Starting Run: ix2tgrsw with config:
wandb: 	cell: RNN
wandb: 	dense_sz: 128
wandb: 	dp: 0.25
wandb: 	edls: 4
wandb: 	emb_sz: 64
wandb: 	epochs: 15
wandb: 	lr: 0.005
wandb: 	nunits: 128


Build Sucessfully
Epoch 1/15
691/691 [==============================] - 235s 326ms/step - loss: 0.9516 - accuracy: 0.3425 - val_loss: 0.8698 - val_accuracy: 0.3672 - _timestamp: 1651576302.0000 - _runtime: 241.0000
Epoch 2/15
691/691 [==============================] - 222s 322ms/step - loss: 0.8829 - accuracy: 0.3781 - val_loss: 0.8440 - val_accuracy: 0.3827 - _timestamp: 1651576525.0000 - _runtime: 464.0000
Epoch 3/15
691/691 [==============================] - 221s 320ms/step - loss: 0.8648 - accuracy: 0.3881 - val_loss: 0.8410 - val_accuracy: 0.3832 - _timestamp: 1651576746.0000 - _runtime: 685.0000
Epoch 4/15
691/691 [==============================] - 223s 323ms/step - loss: 0.8671 - accuracy: 0.3861 - val_loss: 0.8543 - val_accuracy: 0.3733 - _timestamp: 1651576969.0000 - _runtime: 908.0000
Epoch 5/15
691/691 [==============================] - 223s 322ms/step - loss: 0.8704 - accuracy: 0.3838 - val_loss: 0.8482 - val_accuracy: 0.3818 - _timestamp: 1651577192.0000 - _runtime: 1131.0

...

wandb: Agent Starting Run: lndl059l with config:
wandb: 	cell: LSTM
wandb: 	dense_sz: 128
wandb: 	dp: 0.4
wandb: 	edls: 2
wandb: 	emb_sz: 32
wandb: 	epochs: 20
wandb: 	lr: 0.001
wandb: 	nunits: 256


Build Sucessfully
Epoch 1/20
691/691 [==============================] - 63s 66ms/step - loss: 0.9213 - accuracy: 0.3736 - val_loss: 0.6295 - val_accuracy: 0.5375 - _timestamp: 1651588647.0000 - _runtime: 76.0000
Epoch 2/20
691/691 [==============================] - 40s 58ms/step - loss: 0.4735 - accuracy: 0.6466 - val_loss: 0.3219 - val_accuracy: 0.7410 - _timestamp: 1651588687.0000 - _runtime: 116.0000
Epoch 3/20
691/691 [==============================] - 39s 57ms/step - loss: 0.2712 - accuracy: 0.7874 - val_loss: 0.2250 - val_accuracy: 0.8178 - _timestamp: 1651588726.0000 - _runtime: 155.0000
Epoch 4/20
691/691 [==============================] - 39s 57ms/step - loss: 0.1996 - accuracy: 0.8409 - val_loss: 0.1976 - val_accuracy: 0.8383 - _timestamp: 1651588766.0000 - _runtime: 195.0000
Epoch 5/20
691/691 [==============================] - 40s 57ms/step - loss: 0.1629 - accuracy: 0.8696 - val_loss: 0.1832 - val_accuracy: 0.8490 - _timestamp: 1651588805.0000 - _runtime: 234.0000
Epoch 6/

accuracy,▁▄▆▆▇▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▇▇████████████████
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂
accuracy,0.97453
best_epoch,13
best_val_accuracy,0.86656
epoch,19
loss,0.02986
val_accuracy,0.86408


wandb: Agent Starting Run: 4fjg9ufz with config:
wandb: 	cell: RNN
wandb: 	dense_sz: 512
wandb: 	dp: 0.25
wandb: 	edls: 4
wandb: 	emb_sz: 32
wandb: 	epochs: 15
wandb: 	lr: 0.005
wandb: 	nunits: 512


Build Sucessfully
Epoch 1/15
691/691 [==============================] - 283s 396ms/step - loss: 1.3216 - accuracy: 0.2481 - val_loss: 1.1467 - val_accuracy: 0.2376 - _timestamp: 1651589760.0000 - _runtime: 290.0000
Epoch 2/15
691/691 [==============================] - 271s 392ms/step - loss: 1.1670 - accuracy: 0.2343 - val_loss: 1.1098 - val_accuracy: 0.2364 - _timestamp: 1651590032.0000 - _runtime: 562.0000
Epoch 3/15
691/691 [==============================] - 268s 389ms/step - loss: 1.2137 - accuracy: 0.1767 - val_loss: 1.1769 - val_accuracy: 0.1646 - _timestamp: 1651590300.0000 - _runtime: 830.0000
Epoch 4/15
691/691 [==============================] - 267s 387ms/step - loss: 1.2321 - accuracy: 0.1511 - val_loss: 1.1920 - val_accuracy: 0.1586 - _timestamp: 1651590567.0000 - _runtime: 1097.0000
Epoch 5/15
691/691 [==============================] - 266s 385ms/step - loss: 1.2378 - accuracy: 0.1527 - val_loss: 1.1878 - val_accuracy: 0.1551 - _timestamp: 1651590833.0000 - _runtime: 1363.

In [ ]:
wandb.agent(sweep_id, train,entity="pandp",project = 'CS6910-AS3')

wandb: Agent Starting Run: zb1j4467 with config:
wandb: 	cell: GRU
wandb: 	dense_sz: 512
wandb: 	dp: 0.25
wandb: 	edls: 3
wandb: 	emb_sz: 64
wandb: 	epochs: 20
wandb: 	lr: 0.001
wandb: 	nunits: 512
wandb: Currently logged in as: pranab045 (pandp). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Build Sucessfully
Epoch 1/20
691/691 [==============================] - 67s 60ms/step - loss: 0.6806 - accuracy: 0.5216 - val_loss: 0.2827 - val_accuracy: 0.7660 - _timestamp: 1651649708.0000 - _runtime: 83.0000
Epoch 2/20
691/691 [==============================] - 29s 41ms/step - loss: 0.2122 - accuracy: 0.8293 - val_loss: 0.1827 - val_accuracy: 0.8488 - _timestamp: 1651649736.0000 - _runtime: 111.0000
Epoch 3/20
691/691 [==============================] - 28s 41ms/step - loss: 0.1409 - accuracy: 0.8843 - val_loss: 0.1696 - val_accuracy: 0.8587 - _timestamp: 1651649765.0000 - _runtime: 140.0000
Epoch 4/20
691/691 [==============================] - 28s 41ms/step - loss: 0.1054 - accuracy: 0.9119 - val_loss: 0.1669 - val_accuracy: 0.8639 - _timestamp: 1651649793.0000 - _runtime: 168.0000
Epoch 5/20
691/691 [==============================] - 28s 40ms/step - loss: 0.0833 - accuracy: 0.9297 - val_loss: 0.1686 - val_accuracy: 0.8711 - _timestamp: 1651649821.0000 - _runtime: 196.0000
Epoch 6/

accuracy,▁▆▇▇▇███████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▇▇█████████████████
val_loss,█▂▁▁▁▂▂▂▃▄▄▄▄▄▅▅▅▅▆▅
accuracy,0.97251
best_epoch,6
best_val_accuracy,0.87123
epoch,19
loss,0.0318
val_accuracy,0.86739


wandb: Agent Starting Run: y8zacvp1 with config:
wandb: 	cell: LSTM
wandb: 	dense_sz: 512
wandb: 	dp: 0.25
wandb: 	edls: 3
wandb: 	emb_sz: 64
wandb: 	epochs: 15
wandb: 	lr: 0.001
wandb: 	nunits: 128


...

wandb: Agent Starting Run: f8nvq1p4 with config:
wandb: 	cell: LSTM
wandb: 	dense_sz: 128
wandb: 	dp: 0.4
wandb: 	edls: 4
wandb: 	emb_sz: 128
wandb: 	epochs: 15
wandb: 	lr: 0.001
wandb: 	nunits: 256


Build Sucessfully
Epoch 1/15
691/691 [==============================] - 72s 66ms/step - loss: 0.9123 - accuracy: 0.3719 - val_loss: 0.6133 - val_accuracy: 0.5343 - _timestamp: 1651662509.0000 - _runtime: 90.0000
Epoch 2/15
691/691 [==============================] - 36s 52ms/step - loss: 0.5006 - accuracy: 0.6257 - val_loss: 0.3509 - val_accuracy: 0.7300 - _timestamp: 1651662546.0000 - _runtime: 127.0000
Epoch 3/15
691/691 [==============================] - 35s 51ms/step - loss: 0.3015 - accuracy: 0.7660 - val_loss: 0.2435 - val_accuracy: 0.8051 - _timestamp: 1651662581.0000 - _runtime: 162.0000
Epoch 4/15
691/691 [==============================] - 35s 51ms/step - loss: 0.2108 - accuracy: 0.8334 - val_loss: 0.1975 - val_accuracy: 0.8398 - _timestamp: 1651662616.0000 - _runtime: 197.0000
Epoch 5/15
691/691 [==============================] - 36s 53ms/step - loss: 0.1632 - accuracy: 0.8690 - val_loss: 0.1843 - val_accuracy: 0.8510 - _timestamp: 1651662653.0000 - _runtime: 234.0000
Epoch 6/

accuracy,▁▄▆▆▇▇▇▇███████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁
val_accuracy,▁▅▇▇███████████
val_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▂▂
accuracy,0.96728
best_epoch,10
best_val_accuracy,0.86797
epoch,14
loss,0.03799
val_accuracy,0.86263


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4f7k7arr with config:
wandb: 	cell: GRU
wandb: 	dense_sz: 512
wandb: 	dp: 0.4
wandb: 	edls: 3
wandb: 	emb_sz: 32
wandb: 	epochs: 20
wandb: 	lr: 0.005
wandb: 	nunits: 128


Build Sucessfully
Epoch 1/20
691/691 [==============================] - 54s 48ms/step - loss: 0.6410 - accuracy: 0.5451 - val_loss: 0.3136 - val_accuracy: 0.7460 - _timestamp: 1651663128.0000 - _runtime: 70.0000
Epoch 2/20
351/691 [==============>...............] - ETA: 12s - loss: 0.2971 - accuracy: 0.7633

In [ ]:
wandb.agent(sweep_id, train,entity="pandp",project = 'CS6910-AS3')

wandb: Agent Starting Run: 798ze0ts with config:
wandb: 	cell: LSTM
wandb: 	dense_sz: 512
wandb: 	dp: 0
wandb: 	edls: 3
wandb: 	emb_sz: 64
wandb: 	epochs: 15
wandb: 	lr: 0.005
wandb: 	nunits: 256


Build Sucessfully
Epoch 1/15
691/691 [==============================] - 95s 96ms/step - loss: 0.6280 - accuracy: 0.5569 - val_loss: 0.2440 - val_accuracy: 0.7991 - _timestamp: 1651673717.0000 - _runtime: 114.0000
Epoch 2/15
691/691 [==============================] - 58s 84ms/step - loss: 0.1887 - accuracy: 0.8472 - val_loss: 0.1734 - val_accuracy: 0.8547 - _timestamp: 1651673775.0000 - _runtime: 172.0000
Epoch 3/15
691/691 [==============================] - 58s 84ms/step - loss: 0.1303 - accuracy: 0.8921 - val_loss: 0.1558 - val_accuracy: 0.8709 - _timestamp: 1651673833.0000 - _runtime: 230.0000
Epoch 4/15
691/691 [==============================] - 57s 83ms/step - loss: 0.1023 - accuracy: 0.9143 - val_loss: 0.1602 - val_accuracy: 0.8708 - _timestamp: 1651673891.0000 - _runtime: 288.0000
Epoch 5/15
691/691 [==============================] - 58s 84ms/step - loss: 0.0845 - accuracy: 0.9285 - val_loss: 0.1640 - val_accuracy: 0.8742 - _timestamp: 1651673949.0000 - _runtime: 346.0000
Epoch 6

accuracy,▁▆▇▇████
epoch,▁▂▃▄▅▆▇█
loss,█▃▂▂▁▁▁▁
val_accuracy,▁▆██████
val_loss,█▂▁▁▂▂▃▄
accuracy,0.95062
best_epoch,5
best_val_accuracy,0.87547
epoch,7
loss,0.05669
val_accuracy,0.8727


wandb: Agent Starting Run: t4mb09bc with config:
wandb: 	cell: RNN
wandb: 	dense_sz: 128
wandb: 	dp: 0.4
wandb: 	edls: 2
wandb: 	emb_sz: 256
wandb: 	epochs: 20
wandb: 	lr: 0.001
wandb: 	nunits: 512


Build Sucessfully
Epoch 1/20
 11/691 [..............................] - ETA: 2:39 - loss: 1.4025 - accuracy: 0.1497

In [9]:
train = seq2seqModel(Layer = 'LSTM',
                      nunits=128,
                      encl=2,
                      decl=2,
                      embds = 32,
                      dense_size=64,
                      dropout=0.25)

In [10]:
train.compile(optimizer = Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
train.fit([train_encoder_input,train_decoder_input],train_decoder_target,
             batch_size=64,
             validation_data = ([val_encoder_input,val_decoder_input],val_decoder_target),
             epochs=2)

Epoch 1/2
691/691 [==============================] - 66s 56ms/step - loss: 0.9693 - accuracy: 0.3467 - val_loss: 0.7349 - val_accuracy: 0.4616
Epoch 2/2
691/691 [==============================] - 29s 42ms/step - loss: 0.6302 - accuracy: 0.5461 - val_loss: 0.4780 - val_accuracy: 0.6300
